In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import torch
from torchvision import datasets,transforms,models
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import *

import time
import json
import copy
import os
import glob

from PIL import Image

torch.backends.cudnn.enabled = False
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
# torch.backends.cudnn.enabled

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [0]:
# Tansform with data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.5, contrast=0, saturation=0.2, hue=0.1),
        #transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.3853, 0.3780, 0.3631], [0.1965, 0.1920, 0.1924])
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.CenterCrop(299),
        transforms.ColorJitter(brightness=0.5, contrast=0, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.3853, 0.3780, 0.3631], [0.1965, 0.1920, 0.1924])
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'test': transforms.Compose([
        transforms.Resize(256),                                  
        transforms.CenterCrop(224),
        #transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.3473, 0.3418, 0.3290],[0.1896, 0.1837, 0.1802])
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [0]:
#Loading in the dataset

train_dir = "/content/drive/My Drive/Colab Notebooks/535_Project/data/train"  #path to data
label_dir = 'names.csv'

batch_size=32

dataset = datasets.ImageFolder(train_dir,transform=data_transforms['train'])

# splitting data
valid_size  = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size
dataset_sizes = {'train': train_size, 'val': valid_size}

train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

# Loading datasets into dataloader 
dataloaders = {'train': DataLoader(train_dataset, batch_size = batch_size, shuffle = True),
              'val': DataLoader(valid_dataset, batch_size = batch_size, shuffle = True)}



print("Total Number of Samples: ",len(dataset))
print("Number of Samples in Train: ",len(train_dataset))
print("Number of Samples in Valid: ",len(valid_dataset))
print("Number of Classes: ",len(dataset.classes))

print(dataset.classes[0])

In [0]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  #change this to the desired number of classes (in this case 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()

=
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 4 epochs
sched = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [0]:
def train_model(model, criterion, optimizer, sched, num_epochs=5,device='cuda'):
    start = time.time()
    train_results = []
    valid_results = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':            
              model.train()  # Set model to training mode
            else:
              model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        #sched.step()
                        loss.backward()
                        
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            # calculate average time over an epoch
            #elapshed_epoch = time.time() - start/            
            #print('Epoch {}/{} - completed in: {:.0f}m {:.0f}s'.format(epoch+1, num_epochs,elapshed_epoch // 60, elapshed_epoch % 60))
            
            if(phase == 'train'):
              train_results.append([epoch_loss,epoch_acc])
            if(phase == 'val'):
              #sched.step(epoch_acc)
              valid_results.append([epoch_loss,epoch_acc])
                                   
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model (Early Stopping) and Saving our model, when we get best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())       
                model_save_name = "ResNet50.pt"
                path = F"/content/drive/My Drive/Colab Notebooks/535_Project/data/{model_save_name}"
                torch.save(model.state_dict(), path)        

        print()

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    #load best model weights
    model.load_state_dict(best_model_wts)
    
    return model,train_results,valid_results


In [0]:
# Train new data

epochs = 20
model.to(device)
model,train_results,valid_results = train_model(model, criterion, optimizer, sched, epochs)

In [0]:
# Load saved model:
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/535_Project/data/ResNet50_color.pt'))
model.to(device)

In [0]:
import time
import os
import cv2
import copy
from tqdm import tqdm
import csv
import shutil
from PIL import ImageFile



#Align paths
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path



test_dir = "/content/drive/My Drive/Colab Notebooks/535_Project/data/test_original"
# image_dataset = datasets.ImageFolder(test_dir,transform=data_transforms['test'])
image_dataset = ImageFolderWithPaths(test_dir, transform=data_transforms['test'])
  
testloader = torch.utils.data.DataLoader(image_dataset, batch_size=64,
                                             shuffle=True, num_workers=2)
  
# class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
#test network on test data

from pathlib import Path

batch_size = 10
output_list = []
image_names = []

for i, (images, labels, paths) in enumerate(testloader, 0):
    images = images.to(device)
    outputs = model.forward(images)
    _, preds = torch.max(outputs, 1)

    for i in range(len(paths)):
        filename = paths[i]
        filename = (Path(filename).parts[-2] + '/' + Path(filename).stem)
        filename= filename[:-6]
        label = preds[i].item()
        image_names.append(filename)
        output_list.append(str(label))


In [0]:
# Save to csv
df = pd.DataFrame({'guid/image': image_names, 'label': output_list})
pd.set_option('display.max_colwidth', -1)
=
df.to_csv('/content/drive/My Drive/Colab Notebooks/535_Project/Predictions/predictions_ResNet_v4.csv')